In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sb 
import matplotlib.pyplot as plt
import chardet 
from ast import literal_eval

In [2]:
df = pd.read_csv('movies-metadata.csv', encoding = 'utf-8', low_memory=False)
#display all column 
pd.set_option('display.max_columns', None)

In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [4]:
'''def json_to_arr(cell, wanted = "name"): 
    cell = literal_eval(cell)
    if cell == [] or (isinstance(cell, float) and cell.isna()):
        return np.nan
    result = []
    counter = 0
    for element in cell:
        if counter < 3:
            result.append(element[wanted])
            counter += 1
        else:
            break
    return result[:3]'''

'def json_to_arr(cell, wanted = "name"): \n    cell = literal_eval(cell)\n    if cell == [] or (isinstance(cell, float) and cell.isna()):\n        return np.nan\n    result = []\n    counter = 0\n    for element in cell:\n        if counter < 3:\n            result.append(element[wanted])\n            counter += 1\n        else:\n            break\n    return result[:3]'

In [6]:
# make a yes or no colum if movie is in a collection 
df['belongs_to_collection'] = df['belongs_to_collection'].fillna(0)
collection = []
for i in df['belongs_to_collection'] : 
    if i == 0: 
        collection.append('no')
    else : 
        collection.append('yes')
#drop unwanted columns         
df.drop(['belongs_to_collection','adult','tagline','video','homepage','overview','poster_path'], axis = 1 , inplace= True) 

# leave only the relased movies in the data frame         
df['status'] = df.status.value_counts().index[0]
df['collection'] = collection 

#drop duplicates, change to string while droping cause including lists, otherwise comes an Error 
df.loc[df.astype(str).drop_duplicates().index];

#change time column to Datetime type 
df['release_date']= pd.to_datetime(df.release_date, format='%Y-%M-%d', errors = 'ignore')

In [7]:
from ast import literal_eval
# get 'name' from the sting dictionary like columns.

df['production_countries'] = df['production_countries'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df['production_companies'] = df['production_companies'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

# this columsn was removed before, a new yes or no column was created, named collection. 
#df['belongs_to_collection'] = df['belongs_to_collection'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

df['spoken_languages'] = df['spoken_languages'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df['genres'] = df['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [8]:
#fill Nan values with 0 
df = df.fillna(0)

In [9]:
#check final table 
df.head(2)

,budget,genres,id,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,collection
0,30000000,"[Animation, Comedy, Family]",862,tt0114709,en,Toy Story,21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,Toy Story,7.7,5415.0,yes
1,65000000,"[Adventure, Fantasy, Family]",8844,tt0113497,en,Jumanji,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Jumanji,6.9,2413.0,no


In [10]:
df2 = df[['revenue','runtime']]
df2.describe()

,revenue,runtime
count,4.546600e+04,45466.000000
mean,1.120787e+07,93.583711
std,6.432813e+07,38.956167
min,0.000000e+00,0.000000
25%,0.000000e+00,85.000000
50%,0.000000e+00,95.000000
75%,0.000000e+00,107.000000
max,2.787965e+09,1256.000000


In [11]:
#save as csv file 
df.to_csv('Spectacular_Studios.csv', index = False)